"""
# 🎵 Generación de Melodías Condicionadas por Acordes (CMT Transformer)
Este notebook guía el proceso de generación de melodías nuevas a partir de acordes, usando el modelo CMT entrenado.

Pasos:
1️⃣ Cargar un `.pkl` de referencia
2️⃣ Modificar la progresión de acordes
3️⃣ Convertir datos a tensores
4️⃣ Realizar la inferencia
5️⃣ Convertir la salida a MIDI y escuchar la melodía
"""

In [1]:
import pickle
import numpy as np
import torch
import pretty_midi
import matplotlib.pyplot as plt
import sys
sys.path.append("../src")  # Ajusta el path según la ubicación del notebook
from model import ChordConditionedMelodyTransformer  # ✅ Asegúrate de que el path sea correcto


In [2]:

# 📂 Ruta del archivo .pkl de referencia
PKL_PATH = "/home/cepatinog/smc-assignments/final_project/my_jazz_project/data/pkl_files/instance_pkl_8bars_fpb16_48p_12keys/eval/CharlieParker_DonnaLee_FINAL.mid/CharlieParker_DonnaLee_FINAL.mid_00_+0_00.pkl"  # Cambia esto a un archivo válido

In [3]:
# 📌 1️⃣ Cargar el archivo .pkl
with open(PKL_PATH, "rb") as f:
    data = pickle.load(f)
    print("✅ .pkl cargado correctamente con las claves:", data.keys())



✅ .pkl cargado correctamente con las claves: dict_keys(['pitch', 'rhythm', 'chord'])


In [4]:
# 🔄 2️⃣ Modificar la progresión de acordes
new_chord_progression = np.array([
    [1, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],  # C
    [0, 0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0],  # G
    [0, 0, 0, 0, 1, 0, 0, 0, 0, 0, 0, 0],  # Am
    [0, 0, 0, 0, 0, 0, 1, 0, 0, 0, 0, 0]   # F
])

# Ajustar longitud
data["chord"][:len(new_chord_progression)] = new_chord_progression

print("🎶 Nueva progresión de acordes aplicada al .pkl")


🎶 Nueva progresión de acordes aplicada al .pkl


/home/cepatinog/miniconda3/envs/cmt_env/lib/python3.10/site-packages/scipy/sparse/_index.py:210: SparseEfficiencyWarning: Changing the sparsity structure of a csc_matrix is expensive. lil and dok are more efficient.
  self._set_arrayXarray(i, j, x)


In [5]:
# 🧩 3️⃣ Convertir a tensores para la inferencia
device = "cuda" if torch.cuda.is_available() else "cpu"

chord_dense = data["chord"].toarray() if hasattr(data["chord"], "toarray") else data["chord"]
chord_tensor = torch.tensor(chord_dense, dtype=torch.float32).unsqueeze(0).to(device)
prime_pitch = torch.tensor(data["pitch"], dtype=torch.long).unsqueeze(0).to(device)
prime_rhythm = torch.tensor(data["rhythm"], dtype=torch.long).unsqueeze(0).to(device)

In [6]:
# 📥 4️⃣ Realizar la inferencia con el modelo
CHECKPOINT_PATH = "/home/cepatinog/smc-assignments/final_project/my_jazz_project/results/idx002/model/checkpoint_60.pth.tar"  # Ajusta el path
model_config = {
    "num_pitch": 50,
    "frame_per_bar": 16,
    "num_bars": 8,
    "chord_emb_size": 128,
    "pitch_emb_size": 256,
    "hidden_dim": 512,
    "key_dim": 512,
    "value_dim": 512,
    "input_dropout": 0.2,
    "layer_dropout": 0.2,
    "attention_dropout": 0.2,
    "num_layers": 8,
    "num_heads": 16
}

checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)
model = ChordConditionedMelodyTransformer(**model_config).to(device)
model.load_state_dict(checkpoint["model"])
model.eval()


# Verificar la longitud de prime_rhythm
max_len = model.max_len  # Tomamos el max_len del modelo

if prime_rhythm.size(1) > max_len:
    print(f"⚠️ Ajustando la longitud: prime_rhythm ({prime_rhythm.size(1)}) > max_len ({max_len})")
    prime_rhythm = prime_rhythm[:, :max_len]
    prime_pitch = prime_pitch[:, :max_len]
elif prime_rhythm.size(1) < max_len:
    pad_length = max_len - prime_rhythm.size(1)
    print(f"ℹ️ Padding: Agregando {pad_length} ceros a prime_rhythm y prime_pitch")
    pad_rhythm = torch.zeros([1, pad_length], dtype=torch.long).to(prime_rhythm.device)
    pad_pitch = torch.ones([1, pad_length], dtype=torch.long).to(prime_pitch.device) * (model.num_pitch - 1)

    prime_rhythm = torch.cat([prime_rhythm, pad_rhythm], dim=1)
    prime_pitch = torch.cat([prime_pitch, pad_pitch], dim=1)

print(f"✅ Longitud final de prime_rhythm: {prime_rhythm.size(1)}")

with torch.no_grad():
    generated = model.sampling(prime_rhythm, prime_pitch, chord_tensor, topk=5)

pitch_out = generated["pitch"].cpu().numpy()
rhythm_out = generated["rhythm"].cpu().numpy()

print("🎼 Melodía generada con éxito!")

/tmp/ipykernel_217214/3931939090.py:19: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load(CHECKPOINT_PATH, map_location=device)


⚠️ Ajustando la longitud: prime_rhythm (129) > max_len (128)
✅ Longitud final de prime_rhythm: 128
🎼 Melodía generada con éxito!


In [8]:
print(pitch_out)
print(rhythm_out)

[[49 49 49 49 49 15 17 48 20 19 17 16 15 13 48 13 15 48 12 13 15 48 48 48
  49 49 49 49 49 49 49 49 12 19 48 17 14 10 12 48 49 49 49 49 49 49 49 49
  49 49 49 49 13 17 25 27 26 25 24 22 21 20 19 18 17 16 48 15 48 48 49 49
  49 13 48 12 49 49 49 49 22 18 15 48 20 48 16 20 24 27 48 25 48 48 48 48
  30 48 28 23 27 25 48 49 49 49 49 22 24 48 27 25 24 22 21 48 15 48 17 19
  18 17 15 14 17 20 24 18]]
[[0 0 0 0 0 2 2 1 2 2 2 2 2 2 1 2 2 1 2 2 2 1 1 1 0 0 0 0 0 0 0 0 2 2 1 2
  2 2 2 1 0 0 0 0 0 0 0 0 0 0 0 0 2 2 2 2 2 2 2 2 2 2 2 2 2 2 1 2 1 1 0 0
  0 2 1 2 0 0 0 0 2 2 2 1 2 1 2 2 2 2 1 2 1 1 1 1 2 1 2 2 2 2 1 0 0 0 0 2
  2 1 2 2 2 2 2 1 2 1 2 2 2 2 2 2 2 2 2 2]]
